In [ ]:
!pip install meteocalc

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#!cp /content/drive/MyDrive/ashrae.zip ashrae.zip
#!unzip -q ashrae.zip

Mounted at /content/drive


In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.figure_factory as ff

from IPython.core.pylabtools import figsize
from plotly.offline import init_notebook_mode,iplot,plot
%matplotlib inline
init_notebook_mode(connected=True)
plt.style.use("ggplot")

from sklearn.utils import shuffle
from sklearn.preprocessing import  LabelEncoder

from keras.models import load_model
import keras.backend as K

# importer les fichiers .py sur le notebook
%run utils.py
%run feature_enginering.py

ERROR:root:File `'utils.py'` not found.
ERROR:root:File `'feature_enginering.py'` not found.


In [ ]:
KWH_TO_KBTU = 0.2931

building = pd.read_csv('/content/building_metadata.csv')
weather_train = pd.read_csv('/content/weather_train.csv')
train = pd.read_csv('/content/train.csv')

In [ ]:
building = reduce_mem_usage(building)
weather_train  = reduce_mem_usage(weather_train)
train = reduce_mem_usage(train)

In [ ]:
building = prepare_building_data(building)
weather_train = prepare_weather_data(weather_train)
train  = time_features(train)

In [ ]:
train = join_data(train , weather_train , building)

train['month_group'] = train['month_group'] - 1
train = train.drop(['meter_reading'] , axis= 1)
train = reduce_mem_usage(train)

In [ ]:
def join_data(data , weather_data , building_data):

    data = data.merge(building_data, on = 'building_id' , how='left')
    data = data.merge(weather_data , on = ['site_id','timestamp'], how= 'left')
    data = shuffle(data, random_state=0)

    return data

In [ ]:
cols_to_drop = ['year_built' , 'floor_count' , 'timestamp']

cols_to_scale = ['square_feet' , 'air_temperature' , 'dew_temperature' ,
                 'wind_speed', 'cloud_coverage', 'sea_level_pressure', 'wind_direction',
                 'relative_humidity', 'feels_like']

cols_to_encode = ['primary_use' , 'city' , 'country']

cols_to_fill_median = ['air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
                       'sea_level_pressure','wind_direction' , 'wind_speed', 'relative_humidity' ,
                       'feels_like']

cols_to_fill_categ = ['city' , 'country' , 'is_holiday']


Filler = Data_Filler(cols_to_fill_median,
                     cols_to_fill_categ)

Filler.fit(train)
train = Filler.transform(train)

ScalerEncoder = Data_Scaler_Encoder(cols_to_scale ,
                                    cols_to_drop , 
                                    cols_to_encode)
ScalerEncoder.fit(train)
train = ScalerEncoder.transform(train)

del train, weather_train
gc.collect()

171

In [ ]:
test = pd.read_csv('/content/test.csv')
weather_test = pd.read_csv('/content/weather_test.csv')

In [ ]:
weather_test  = reduce_mem_usage(weather_test)
test = reduce_mem_usage(test)

In [ ]:
weather_test = prepare_weather_data(weather_test)
test = time_features(test)

In [ ]:
def rmse(y_true, y_pred):
    """ root_mean_squared_error """
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

model = load_model('/content/drive/MyDrive/model_ann.h5',
                   custom_objects = {'rmse' : rmse })

model.compile('adam', loss='mse')

In [ ]:
categorical_vars = ['building_id' , 'meter', 'hour', 'dayofweek' , 'month_group', 'site_id' , 
                    'primary_use', 'city', 'country', 'is_holiday']
                    
numerical_vars = ['square_feet', 'air_temperature', 'cloud_coverage', 'dew_temperature',
                  'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed', 
                  'relative_humidity' , 'feels_like']

co_vars = categorical_vars + numerical_vars

In [ ]:
step_size = 100000
nrows = test.shape[0]
num_steps = nrows//step_size
print('nombre d\'iterations' , num_steps)

row_id = np.array([])
meter_values = np.array([])
y_pred = np.array([])

for j in range(num_steps+1):
    start_idx = j*step_size
    end_idx = (j+1)*step_size
    if j == num_steps : 
        start_idx = j*step_size
        end_idx = nrows
      
    test_tmp = test.iloc[start_idx:end_idx]
    
    test_tmp = join_data(test_tmp , weather_test , building)
    test_tmp = Filler.transform(test_tmp)
    test_tmp = ScalerEncoder.transform(test_tmp)
    test_tmp['month_group'] = test_tmp['month_group'] - 1
    
    row_id_tmp = test_tmp['row_id'].values
    row_id = np.concatenate((row_id,row_id_tmp))

    test_tmp = test_tmp.drop(['row_id'] , axis= 1)

    meter_values_tmp = test_tmp['meter'].values
    meter_values = np.concatenate((meter_values,meter_values_tmp))

    test_tmp = test_tmp[co_vars].values
    test_tmp = [ test_tmp[:,i].flatten() for i in range(len(test_tmp[0,:])) ]
    
    predictions = model.predict(test_tmp)
    predictions = predictions.reshape(predictions.shape[0])

    y_pred = np.concatenate((y_pred,predictions))
    if j%50==0:
        print('iteration' , j)

nombre d'iterations 416
iteration 0
iteration 50
iteration 100
iteration 150
iteration 200
iteration 250
iteration 300
iteration 350
iteration 400


In [ ]:
del building, weather_test,test

gc.collect()

NameError: ignored

In [ ]:
pred_final = {'row_id': row_id, 'meter' : meter_values , 'meter_reading': y_pred}
pred_final = pd.DataFrame(data = pred_final)

In [ ]:
pred_final['meter_reading'] = np.exp(pred_final['meter_reading']) - 1
pred_final.loc[(pred_final.meter == 0),'meter_reading'] = pred_final['meter_reading'] / KWH_TO_KBTU # convertir du kwh vers kbtu

In [ ]:
pred_final = pred_final.sort_values(by=['row_id'])
pred_final = pred_final.drop(['meter'] , axis = 1)
pred_final.loc[pred_final.meter_reading < 0] = 0


In [ ]:
compression_opts = dict(method='zip',archive_name='out.csv')  

pred_final.to_csv('out.zip', index=False,compression=compression_opts)  

In [ ]:
!cp out.zip /content/drive/MyDrive/out.zip

In [ ]:

pred_final.loc[pred_final.meter_reading < 0] = 0
pred_final

,row_id,meter_reading
52860,0.0,0.000000
33031,1.0,0.021453
83905,0.0,0.000000
15517,3.0,0.040544
46516,4.0,0.060824
...,...,...
41640965,41697595.0,5.449270
41659597,41697596.0,3.694571
41675115,41697597.0,5.863081
41629031,41697598.0,146.382700


In [ ]:
#!cp /content/drive/MyDrive/out.zip out.zip
#!unzip -q out.zip

In [ ]:
pred_final = pd.read_csv('/content/out.csv')
pred_final.row_id = pred_final.row_id.astype('int32')

In [ ]:
compression_opts = dict(method='zip',archive_name='out_final.csv')  

pred_final.to_csv('out_final.zip', index=False,compression=compression_opts)

In [ ]:
!cp out_final.zip /content/drive/MyDrive/out_final.zip

In [ ]:
sample = pd.read_csv('/content/sample_submission.csv')

In [ ]:
sample.meter_reading

0           0
1           0
2           0
3           0
4           0
           ..
41697595    0
41697596    0
41697597    0
41697598    0
41697599    0
Name: meter_reading, Length: 41697600, dtype: int64